In [430]:
from math import ceil
import time
import re
import ast
import sys
import urllib
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from urllib.error import URLError
import requests
from itertools import cycle
import string

In [431]:
header1 = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) '
           'AppleWebKit 537.36 (KHTML, like Gecko) Chrome',
           'Accept': 'text/html,application/xhtml+xml,application/xml;'
           'q=0.9,image/webp,*/*;q=0.8'}
header2 = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) '
           'AppleWebKit 537.36 (KHTML, like Gecko) Chrome',
           'Accept': 'text/html,application/xhtml+xml,application/xml;'
           'q=0.9,image/webp,*/*;q=0.8'}
header3 = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
           'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
           'Accept': 'text/html,application/xhtml+xml,application/xml;'
           'q=0.9,image/webp,image/apng,*/*;q=0.8'}
header4 = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:62.0) '
           ' Gecko/20100101 Firefox/62.0',
           'Accept': 'text/html,application/xhtml+xml,application/xml;'
           'q=0.9,*/*;q=0.8'}
ua = [header1, header2, header3, header4]

In [591]:
proxies = ['64.130.65.21:50295', '66.191.121.167:33849', '104.236.144.230:8080', 
           '166.141.173.247:49550', '68.250.114.126:52515', '24.173.244.146:21231', 
           '24.229.120.65:32857', '37.224.84.90:8080', '38.88.154.218:50860', 
           '173.17.122.32:44400', '216.75.113.182:39602', '12.183.155.92:55222', 
           '64.19.116.82:60814', '216.215.123.174:8080', '72.252.4.110:38156', 
           '65.173.110.100:43386', '50.224.173.189:8080', '69.174.159.176:8080', 
           '50.245.151.237:48131', '208.117.126.34:46101', '72.35.40.34:8080', 
           '50.224.173.190:8080', '75.109.158.167:54318']

prx_pool = random.sample(proxies, len(proxies))
len(proxies)

23

In [433]:
def proxie_check(proxies):
    default_list = []
    url = 'https://httpbin.org/ip'
    for i in range(0, len(proxies)):
        proxy = proxies[i]
        print(i+1)
        start_time = time.time()
        try:
            response = requests.get(
                url, proxies={"http": proxy, "https": proxy})
            print(response.json())
            print(time.time() - start_time)
        except:
            print("Skipping. Connnection error")
            default_list.append(i+1)
            print(time.time() - start_time)
        print(default_list)
    return default_list
# proxie_check(proxies)

In [558]:
def session_creator(proxy, ua, url):
    header = random.sample(ua, 1)[0]
    session = requests.Session()
    session.proxies = {"http": proxy, "https": proxy}
    req = session.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'lxml')
    return soup

In [435]:
def no_proxy_session(ua, url):
    header = random.sample(ua, 1)[0]
    session = requests.Session()
    req = session.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'lxml')
    return soup

In [436]:
zip_codes = pd.read_csv('../Data/zips.csv')
zip_list = zip_codes['zip'].tolist()
zip_list = [num for num in zip_list if str(num) != 'nan']

In [437]:
# rand_zip_list_ap = random.sample(zip_list, len(zip_list))
# ap_prp_count_list = []

In [438]:
# rand_zip_sld = random.sample(zip_list, len(zip_list))
# sold_prp_list = []
# ezl = []

In [439]:
def zip_prop_count(zip_list, proxies, prp_list, ua, ezl):
    proxy = random.sample(proxies, 1)[0]
    print(proxies.index(proxy))
    print(proxy)
    for num in zip_list:
        url = 'https://www.redfin.com/zipcode/' + \
            str(num)+'/filter/property-type=house+condo+townhouse,include=sold-1yr,min-price=100k,min-baths=1,include=sold-1yr'
        try:
            start_time = time.time()
            soup = session_creator(proxy, ua, url)
            #all_count = soup.findAll('span', {'data-rf-test-name': 'homes-for-sale'})
            print(num)
            print(len(zip_list))
            all_count = soup.findAll('div', {'class': 'homes summary'})
            if len(str(all_count)) >= 20:
                print(all_count)
                print(time.time() - start_time)
                ezl.append(num)
                prp_list.append(all_count)
                zip_list.remove(num)
                print(len(zip_list)+len(prp_list))
            else:
                print("Captcha!!!!!")
        except:
            print("Skipping. Connnection error")
            proxies.remove(proxy)
            print(len(proxies))
            return prp_list, zip_list, proxies, ezl
    return prp_list, zip_list, proxies, ezl

#ap_prp_count_list, rand_zip_list, proxies = zip_prop_count(rand_zip_list_ap, proxies, ap_prp_count_list, ua)
#sold_prp_list, rand_zip_sld, proxies, ezl = zip_prop_count(rand_zip_sld, proxies, sold_prp_list, ua, ezl)

In [497]:
sold_prp_list = [str(num) for num in sold_prp_list]
sold_prp_list = [num.replace(
    '[<div class="homes summary" data-rf-test-id="homes-description"><span class="showingText">Showing </span>', '') for num in sold_prp_list]
sold_prp_list = [num.replace(
    ' Homes<span class="summarySeparator ">•</span></div>]', '') for num in sold_prp_list]
sold_prp_list = [num.replace('20 of ', '') for num in sold_prp_list]

In [498]:
def strip_count(lst):
    rx_num_homes = r'\d+\shomes'
    rx_zip = r'\d+\sat'
    rx_median = r'\$\d+\.?\w+'
    median_list = []
    zip_list = []
    num_homes_list = []
    for num in lst:
        num = str(num)
        median_ = re.findall(rx_median, num, re.MULTILINE)
        zip_ = re.findall(rx_zip, num, re.MULTILINE)
        num_homes_ = re.findall(rx_num_homes, num, re.MULTILINE)
        median_list.extend([i for i in median_])
        zip_list.extend([i for i in zip_])
        num_homes_list.extend([i for i in num_homes_])

    median_list = [num.replace('$', '').replace(
        'K', ',000').replace('.', ',') for num in median_list]
    i = 0
    while i < len(median_list):
        if len(median_list[i]) == 2:
            median_list[i] = median_list[i].replace('M', ',000,000')
            i += 1
        elif len(median_list[i]) == 4:
            median_list[i] = median_list[i].replace('M', '00,000')
            i += 1
        elif len(median_list[i]) == 5:
            median_list[i] = median_list[i].replace('M', '0,000')
            i += 1
        else:
            i += 1
    zip_list = [re.findall(r'\d+', num, re.MULTILINE)[0] for num in zip_list]
    num_homes_list = [re.findall(r'\d+', num, re.MULTILINE)[0]
                      for num in num_homes_list]

    df = pd.DataFrame(
        data={'zip': zip_list, 'median_price': median_list, 'num_ap_homes': num_homes_list})
    return df

In [499]:
sdf = pd.DataFrame({'zip': ezl, 'sld': sold_prp_list})
apdf = strip_count(ap_prp_count_list)
apdf.to_csv('../Data/apdf.csv')
sdf.to_csv('../Data/sdf.csv')

apdf.index = ap_df['zip']
sdf.index = sdf['zip']

sdf['sld'] = (sdf['sld']).astype(int)
apdf['num_ap_homes'] = (ap_df['num_ap_homes']).astype(int)

sdf['zip'] = (sdf['zip']).astype(int)
apdf['zip'] = (ap_df['zip']).astype(int)

sld_pgs = [ceil(num/20) for num in sdf['sld']]
ap_pgs = [ceil(num/20) for num in apdf['num_ap_homes']]

sdf['pages'] = sld_pgs
apdf['pages'] = ap_pgs

sdf = sdf.sort_values(by='pages')
apdf = apdf.sort_values(by='pages')

sdf_under = sdf[sdf['pages'] <= 18]
sdf_over = sdf[sdf['pages'] > 18]

In [577]:
ap_url_list = []
for num in zip_list:
    num_pages_count = apdf.loc[apdf['zip'] == num].pages.values[0]
    for i in range(1, num_pages_count+1):
        url = 'https://www.redfin.com/zipcode/'+str(num) + \
            '/filter/property-type=house+condo+townhouse,min-baths=1,status=active+pending+contingent/page-' + \
            str(i)
        ap_url_list.append(url)


ap_url_list = [num.replace('/page-1', '') for num in ap_url_list]
ap_url_list = [num.replace('contingent0', 'contingent/page-10')
               for num in ap_url_list]
ap_url_list = [num.replace('contingent1', 'contingent/page-11')
               for num in ap_url_list]
ap_url_list = [num.replace('contingent2', 'contingent/page-12')
               for num in ap_url_list]
ap_url_list = [num.replace('contingent3', 'contingent/page-13')
               for num in ap_url_list]

# main_df = pd.DataFrame(columns=['full_address', 'home_link'])

ap_url_list = sorted(ap_url_list)
print(len(ap_url_list))

446


In [529]:
# sld_under_url_list = []

# for num in sdf_under['zip']:
#     num_pages_count = sdf_under.loc[sdf_under['zip'] == num].pages.values[0]
#     for i in range(1, num_pages_count+1):
#         url = 'https://www.redfin.com/zipcode/'+str(num) + \
#             '/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-' + \
#             str(i)
#         sld_under_url_list.append(url)


# sld_under_url_list = [num.replace('/page-1', '') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr0', '=sold-1yr/page-10') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr1', '=sold-1yr/page-11') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr2', '=sold-1yr/page-12') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr3', '=sold-1yr/page-13') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr4', '=sold-1yr/page-14') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr5', '=sold-1yr/page-15') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr6', '=sold-1yr/page-16') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr7', '=sold-1yr/page-17') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr8', '=sold-1yr/page-18') for num in sld_under_url_list]

# s_main_df = pd.DataFrame(columns=['full_address', 'home_link'])

sld_under_url_list = sorted(sld_under_url_list)
# print(len(sld_under_url_list))

923


In [540]:
e1s = {'full_address': [], 'home_link': []}
e2s = {'full_address': [], 'home_link': []}
e1s['full_address'] == e2s['full_address']

True

In [578]:
def each_page(proxy, ua, url):
    soup = session_creator(proxy, ua, url)
    start_time = time.time()
    time.sleep(random.uniform(0, 1)*4)
    print(time.time()-start_time)
    full_soup = soup.findAll('a', {'class': 'bottom link-override'})
    full_address = [fas['title'] for fas in full_soup]
    home_link = ['https://www.redfin.com' +
                 str(hls.get('href')) for hls in full_soup]
    df = {'full_address': full_address, 'home_link': home_link}
    return df

In [581]:
def links_for_props(proxies, url_list, main_df, ua):
    proxy = random.sample(proxies, 1)[0]
    print(proxies.index(proxy))
    print(proxy)
    i = 0
    while i < len(url_list):
        url = url_list[i]        
        try:
            start_time = time.time()
            print(len(url_list))
            print(url)
            
            data = each_page(proxy, ua, url)
            df = pd.DataFrame(data)
            eds = {'full_address': [], 'home_link': []}
            if data['full_address'] != eds['full_address']:
                main_df = pd.concat([main_df, df])
                url_list.pop(i)
                print(time.time() - start_time)
            else:
                print('Captcha')
                return url_list, main_df
        except:
            print("Skipping. Connnection error")
            proxies.remove(proxy)
            print(len(proxies))

            return url_list, main_df

    return url_list, main_df

In [594]:
a = randint(0,3)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)
time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)

6
50.224.173.190:8080
41
https://www.redfin.com/zipcode/95127/filter/property-type=house+condo+townhouse,min-baths=1,status=active+pending+contingent
1.3094654083251953
Captcha
1
37.224.84.90:8080
41
https://www.redfin.com/zipcode/95127/filter/property-type=house+condo+townhouse,min-baths=1,status=active+pending+contingent
2.209718942642212
Captcha
5
72.35.40.34:8080
41
https://www.redfin.com/zipcode/95127/filter/property-type=house+condo+townhouse,min-baths=1,status=active+pending+contingent
0.7326216697692871
Captcha
5
72.35.40.34:8080
41
https://www.redfin.com/zipcode/95127/filter/property-type=house+condo+townhouse,min-baths=1,status=active+pending+contingent
3.1994664669036865
Captcha
3
65.173.110.100:43386
41
https://www.redfin.com/zipcode/95127/filter/property-type=house+condo+townhouse,min-baths=1,status=active+pending+contingent
0.503619909286499
Captcha
0
24.173.244.146:21231
41
https://www.redfin.com/zipcode/95127/filter/property-type=house+condo+townhouse,min-baths=1,status

ValueError: Sample larger than population or is negative

In [573]:
main_df

,full_address,home_link


In [536]:
# sld_under_url_list = sld_under_url_list[1:]
print(len(sld_under_url_list))

839


In [562]:
sld_under_url_list, s_main_df = links_for_props(
    proxies, sld_under_url_list, s_main_df, ua)

6
37.224.84.90:8080
724
https://www.redfin.com/zipcode/94118/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr
15.772884607315063
723
https://www.redfin.com/zipcode/94118/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-10
12.27623200416565
722
https://www.redfin.com/zipcode/94118/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-11
13.18439793586731
721
https://www.redfin.com/zipcode/94118/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-12
13.3983793258667
720
https://www.redfin.com/zipcode/94118/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-2
49.344985008239746
719
https://www.redfin.com/zipcode/94118/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-3
22.860891342163086
718
https://www.redfin.com/zipcode/94118/filter/property-type=ho

19.188801050186157
670
https://www.redfin.com/zipcode/94124/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-8
13.337065696716309
669
https://www.redfin.com/zipcode/94124/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-9
13.961189985275269
668
https://www.redfin.com/zipcode/94127/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr
12.745570659637451
667
https://www.redfin.com/zipcode/94127/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-10
18.248830556869507
666
https://www.redfin.com/zipcode/94127/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-2
11.66709589958191
665
https://www.redfin.com/zipcode/94127/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-3
16.864686727523804
664
https://www.redfin.com/zipcode/94127/filter/property-type=ho

13.030513525009155
616
https://www.redfin.com/zipcode/94158/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-4
13.71080493927002
615
https://www.redfin.com/zipcode/94158/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-5
12.906861305236816
614
https://www.redfin.com/zipcode/94158/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-6
15.687701940536499
613
https://www.redfin.com/zipcode/94158/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-7
10.032436847686768
612
https://www.redfin.com/zipcode/94158/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-8
14.3046875
611
https://www.redfin.com/zipcode/94158/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-9
12.904346704483032
610
https://www.redfin.com/zipcode/94301/filter/property-type=hous

15.645692110061646
562
https://www.redfin.com/zipcode/94401/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-7
9.363784074783325
561
https://www.redfin.com/zipcode/94401/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-8
18.4793119430542
560
https://www.redfin.com/zipcode/94401/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-9
13.100482940673828
559
https://www.redfin.com/zipcode/94402/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr
13.80682921409607
558
https://www.redfin.com/zipcode/94402/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-10
16.387946128845215
557
https://www.redfin.com/zipcode/94402/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-11
18.20107364654541
556
https://www.redfin.com/zipcode/94402/filter/property-type=house

6.351072788238525
508
https://www.redfin.com/zipcode/94555/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-13
11.575873374938965
507
https://www.redfin.com/zipcode/94555/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-14
11.499136686325073
506
https://www.redfin.com/zipcode/94555/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-15
10.013700008392334
505
https://www.redfin.com/zipcode/94555/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-16
9.936211347579956
504
https://www.redfin.com/zipcode/94555/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-17
11.213463544845581
503
https://www.redfin.com/zipcode/94555/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-2
9.056727886199951
502
https://www.redfin.com/zipcode/94555/filter/propert

...

9.985642194747925
238
https://www.redfin.com/zipcode/95110/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-6
8.772775888442993
237
https://www.redfin.com/zipcode/95110/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-7
23.468796730041504
236
https://www.redfin.com/zipcode/95110/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-8
14.224984645843506
235
https://www.redfin.com/zipcode/95110/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-9
12.035128831863403
234
https://www.redfin.com/zipcode/95111/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr
11.076702356338501
233
https://www.redfin.com/zipcode/95111/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-10
11.19517731666565
232
https://www.redfin.com/zipcode/95111/filter/property-type=hous

17.237971544265747
184
https://www.redfin.com/zipcode/95117/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-2
34.23559522628784
183
https://www.redfin.com/zipcode/95117/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-3
25.959577083587646
182
https://www.redfin.com/zipcode/95117/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-4
23.27587580680847
181
https://www.redfin.com/zipcode/95117/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-5
23.798372745513916
180
https://www.redfin.com/zipcode/95117/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-6
18.198793649673462
179
https://www.redfin.com/zipcode/95117/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-7
20.8095805644989
178
https://www.redfin.com/zipcode/95117/filter/property-type

55.144524574279785
130
https://www.redfin.com/zipcode/95126/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-5
42.293219327926636
129
https://www.redfin.com/zipcode/95126/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-6
32.039567708969116
128
https://www.redfin.com/zipcode/95126/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-7
39.39194130897522
127
https://www.redfin.com/zipcode/95126/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-8
33.560224294662476
126
https://www.redfin.com/zipcode/95126/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-9
31.745256900787354
125
https://www.redfin.com/zipcode/95128/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr
52.477537870407104
124
https://www.redfin.com/zipcode/95128/filter/property-type=hou

13.837615251541138
76
https://www.redfin.com/zipcode/95132/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-12
48.724186420440674
75
https://www.redfin.com/zipcode/95132/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-13
52.627360343933105
74
https://www.redfin.com/zipcode/95132/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-14
23.578075408935547
73
https://www.redfin.com/zipcode/95132/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-15
37.767096519470215
72
https://www.redfin.com/zipcode/95132/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-16
35.95078706741333
71
https://www.redfin.com/zipcode/95132/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-17
20.87750267982483
70
https://www.redfin.com/zipcode/95132/filter/property-typ

18.397449254989624
22
https://www.redfin.com/zipcode/95138/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-9
43.81352138519287
21
https://www.redfin.com/zipcode/95139/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr
13.953850030899048
20
https://www.redfin.com/zipcode/95139/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-2
22.854965448379517
19
https://www.redfin.com/zipcode/95139/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-3
35.39326500892639
18
https://www.redfin.com/zipcode/95139/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-4
19.39294147491455
17
https://www.redfin.com/zipcode/95139/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-5
15.856658458709717
16
https://www.redfin.com/zipcode/95148/filter/property-type=house+condo+

In [566]:
list_ = list(s_main_df.full_address.values)
set([num[-5:] for num in list_])

{' 9412',
 ' 9417',
 '-0000',
 '-1003',
 '-1032',
 '-1037',
 '-1103',
 '-1111',
 '-1128',
 '-1131',
 '-1167',
 '-1205',
 '-1213',
 '-1247',
 '-1249',
 '-1316',
 '-1321',
 '-1325',
 '-1332',
 '-1336',
 '-1347',
 '-1414',
 '-1427',
 '-1446',
 '-1454',
 '-1460',
 '-1503',
 '-1542',
 '-1614',
 '-1622',
 '-1627',
 '-1638',
 '-1639',
 '-1700',
 '-1729',
 '-1733',
 '-1735',
 '-1766',
 '-1783',
 '-1807',
 '-1809',
 '-1837',
 '-1849',
 '-1908',
 '-1912',
 '-1917',
 '-1920',
 '-1934',
 '-1937',
 '-1938',
 '-1939',
 '-1958',
 '-2014',
 '-2057',
 '-2148',
 '-2211',
 '-2220',
 '-2321',
 '-2328',
 '-2332',
 '-2336',
 '-2358',
 '-2407',
 '-2410',
 '-2414',
 '-2429',
 '-2437',
 '-2446',
 '-2504',
 '-2509',
 '-2545',
 '-2581',
 '-2583',
 '-2603',
 '-2606',
 '-2607',
 '-2634',
 '-2700',
 '-2709',
 '-2714',
 '-2715',
 '-2756',
 '-2795',
 '-2808',
 '-2825',
 '-2830',
 '-2832',
 '-2872',
 '-2877',
 '-2909',
 '-2915',
 '-3003',
 '-3010',
 '-3013',
 '-3021',
 '-3034',
 '-3037',
 '-3049',
 '-3121',
 '-3172',


In [569]:
s_main_df

,full_address,home_link
0,"400 Ortega Ave #323, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/400-Or...
1,"1408 Ernestine Ln, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/1408-E...
2,"400-B Ortega Ave #302, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/400-Or...
3,"989 Solana Ct, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/989-So...
4,"49 Showers Dr Unit A331, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/49-Sho...
5,"71 Glenmead Ct, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/71-GLE...
6,"2481 Porterfield Ct, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/2481-P...
7,"992 Boranda Ave, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/992-Bo...
8,"1303 Cuernavaca Circulo, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/1303-C...
9,"2524 Fairbrook Dr, MOUNTAIN VIEW, CA 94040",https://www.redfin.com/CA/Mountain-View/2524-F...


In [572]:
s_main_df.to_csv('../Data/s_main.csv')

In [ ]:
def each_property(url, hdr, proxy):
    soup = session_creator(proxy, ua, url)
#     homefeat_desc = []
#     homefeat_info = []

    home_address_MLS = soup.title.string
    print(home_address_MLS)
    
    price = soup.findAll('div', {'class': 'statsValue'})
    for num in price:
        print(num.span)

    home_full_feat_desc = soup.findAll('span', {'class': 'statsLabel'})
    for desc in home_full_feat_desc:
        print(desc)
#         homefeat_desc.append(str(desc.contents[0]))

    home_full_feat_info = soup.findAll('span', {'class': 'statsValue'})
    for info in home_full_feat_info:
        print(info)
#         homefeat_info.append(str(info.contents[0]))

#     new_dict = dict(zip(homefeat_desc, homefeat_info))
#     new_dict['address'] = str(home_address_MLS)

    home_description = soup.find('p', {'class': 'font-b1'})
    print(home_description.span)

#     new_dict['home_description'] = str(home_description.attrs['content'])
    
    other_home_feats = soup.findAll('span', {'class':"header font-color-gray-light font-weight-roman"})
    print(other_home_feats)
    other_home_feats_vals = soup.findAll('span', {'class':"content font-weight-roman"})
    print(other_home_feats_vals)
    #both are in soup.findAll('div', {'class':"keyDetail font-size-base"})

    # lots of info:
    all_home_feats = soup.findAll('div', {'class'="amenities-container"})
    
    estimate = soup.find('div', {'data-rf-test-name': 'avmValue"'})
    print(estimate)

#     home_hist_src = soup.findAll('td', {'class': 'zsg-sm-hide'})
#     print(home_hist_src)

    prop_hist = soup.findAll('div', {'class': 'basic-table-2'})
    print(prop_hist)
    
    school_info = soup.findAll('div', {'class'="schools-table"})
    print(school_info)

#     school_rating = soup.findAll('div', {'class': 'nearby-schools-rating'})
#     for rating in school_rating:
#         print(rating.span.text)
        
#     school_name_regex = r'school-name notranslate$'
#     school_name = soup.findAll('a', {'class': 'school-name notranslate'})
#     for mane in school_name:
#         print(mane.next_element)

#     school_grades = soup.findAll('div', {'class': 'nearby-schools-grades'})
#     for grade in school_grades:
#         print(grade.text)

#     school_assigned = soup.findAll(
#         'span', {'class': 'assigned-label zsg-fineprint'})

#     for assi in school_assigned:
#         print(assi.text)

#     school_dist = soup.findAll('div', {'class': 'nearby-schools-distance'})
#     for dist in school_dist:
#         print(dist.text)
#     return new_dict

#     discalimer = soup.find('section', {'class': 'zsg-content-section'})
#     print(discalimer)